# Определение теплоты испарения жидкости

In [64]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from scipy.interpolate import UnivariateSpline

def read_files(files):
    data = []
    for f in files:
        data += [pd.read_csv(f, sep=',', skipinitialspace=True)]
    return data

def get_linear(x, y, xerr, yerr):
    a, b = np.polyfit(x, y, deg=1)
    aerr = np.sqrt(1 / (x.__len__() - 1) * (sum([(it - b)**2 for it in y]) 
                                                    / sum([it**2 for it in x]) - a**2))
    syserr = np.sqrt((np.average(xerr) / (max(x) - min(x)))**2
                         + (np.average(yerr) / (max(y) - min(y)))**2)
    a_syserr = a * syserr
    berr = aerr * np.sqrt(np.average([it**2 for it in x]))
    b_syserr = b * syserr
    return ((a, b), (np.sqrt(aerr**2 + a_syserr**2), np.sqrt(berr**2 + b_syserr**2)))

labels = ['Heating', 'Cooling']

[data, cooling] = read_files(['data/data.csv', 'data/cooling.csv'])
colors = ['red', 'green']

plt.figure(figsize=(7, 4))

i = 0
for it in [data, cooling]:
    inverse_T = it['1/T']
    ln_P = it['lnP']
    plt.errorbar(inverse_T, ln_P, xerr=0, yerr=0, fmt=f'{colors[i][0]}.')
    ((a, b), (aerr, berr)) = get_linear(inverse_T, ln_P, [0], [0])
    z = np.linspace(min(inverse_T), max(inverse_T), 1000)
    plt.plot(z, a * z + b, color=colors[i], label=labels[i])
    i += 1
    
# Оформление
plt.xlabel('$1 / T$, K$^{-1}$')
plt.ylabel('$\\ln P$')
plt.grid(linestyle='--')
plt.legend()
plt.savefig('data/lin.pdf')

plt.figure(figsize=(7, 4))

i = 0
for it in [data, cooling]:
    T = it['T']
    P = it['P']
    plt.errorbar(T, P, xerr=0.1, yerr=0.1, fmt=f'{colors[i][0]}.')
    if (i == 0):
        s = UnivariateSpline(T, P, s=5)
    z = np.linspace(min(T), max(T), 1000)
    plt.plot(z, s(z), color=colors[i], label=labels[i])
    i += 1

# Оформление
plt.xlabel('$T$, \\textdegree C')
plt.ylabel('$P$, mm hg')
plt.grid(linestyle='--')
plt.legend()
plt.savefig('data/curve.pdf')
